In [ ]:
# written by Peter Culviner, PhD to enable command-line access through Jupyter
def quickshell(command, print_output=True, output_path=None, return_output=False):
    process_output = subprocess.run(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout = process_output.stdout.decode('utf-8')
    stderr = process_output.stderr.decode('utf-8')
    output_string = f'STDOUT:\n{stdout}\nSTDERR:\n{stderr}\n'
    if print_output:
        print('$ ' + command)
        print(output_string)
    if output_path is not None:
        with open(output_path, 'w') as f:
            f.write(output_string)
    if return_output:
        return stdout, stderr

In [ ]:
# Inputs
main_path = '5enrich_CRP'
sample_list = ['noCRP1','noCRP2','noCRP3',
              'CRP1','CRP2','CRP3']

#main_path = '3enrich_NusAG'
#sample_list = ['noTF1','noTF2','noTF3',
#              'NusA1','NusA2','NusA3',
#              'NusG1','NusG2','NusG3',
#              'NusA_NusG1','NusA_NusG2','NusA_NusG3']

#gDNA_path = 'gDNA'
#sample_list = ['gDNA1','gDNA2','gDNA3']

identifyEnrichedEnds_dir = f'{main_path}/identifyEnrichedEnds'
downsample_dir = f'{identifyEnrichedEnds_dir}/downsampled_R2_bams'
spike_downsample_dir = f'{identifyEnrichedEnds_dir}/downsampled_spike_R2_bams'

bigWig_dir = f'{identifyEnrichedEnds_dir}/bigWigs_vis'

In [ ]:
!mkdir $bigWig_vis_dir

In [ ]:
# Set print_output = True to see command-line output
# Set return_output = False if assigning commmand-line output to a variable
print_output = False
return_output = False

ends_region = 'Eco_Mtb:4641653:9053361'

downsample_stats = pd.read_csv(f'{downsample_dir}/downsampled_depths.csv')
max_spike_counts = downsample_stats['Downsampled_spike_read_counts'].max()

# Generate bigWig files for visualization
for sample in sample_list:
    spike_counts = downsample_stats.loc[downsample_stats['Sample_Name'] == sample].Downsampled_spike_read_counts
    spikeFactor = max_spike_counts / spike_counts
    
    bigwig_plus_command = f'bamCoverage --bam {downsample_dir}/{sample}_downsample.bam ' + \
                          f'--numberOfProcessors {processors} --region {ends_region} ' + \
                          f'--skipNonCoveredRegions --skipNAs --scaleFactor {int(spikeFactor)} ' + \
                          f'--outFileName {bigWig_vis_dir}/{sample}_vis_plus.bw --outFileFormat bigwig ' + \
                          f'--binSize 1 --filterRNAstrand forward --Offset 1' 
    bigwig_minus_command = f'bamCoverage --bam {downsample_dir}/{sample}_downsample.bam ' + \
                           f'--numberOfProcessors {processors} --region {ends_region} '+ \
                           f'--skipNonCoveredRegions --skipNAs --scaleFactor {int(spikeFactor)} ' + \
                           f'--outFileName {bigWig_vis_dir}/{sample}_vis_minus.bw --outFileFormat bigwig ' + \
                           f'--binSize 1 --filterRNAstrand reverse --Offset 1' 
 
    bigwig_plus = quickshell(bigwig_plus_command, print_output = print_output, return_output = return_output)
    print(f'bigWig generation (visualization): {sample} plus done')
    bigwig_minus = quickshell(bigwig_minus_command, print_output = print_output, return_output = return_output)
    print(f'bigWig generation (visualization): {sample} minus done')